In [1]:
import lmdb
import pickle
import glob
import os
import pandas as pd
from torch_geometric.data import Data

In [2]:
train_dir = "oc22/is2re-total/train"
train_files = sorted(glob.glob(os.path.join(train_dir, "data.*.lmdb")))

# all the properties available in the metadata pickle file
columns = [
    "sid",
    "y_relaxed",
    "bulk_id",
    "bulk_symbols",
    "miller_index",
    "traj_id",
    "slab_sid",
    "ads_symbols",
    "nads"
]

# read metadata file for bulk symbol
with open("oc22_metadata.pkl", "rb") as file:
    metadata = pickle.load(file)

all_data = []

for lmdb_file in train_files:
    env = lmdb.open(lmdb_file, readonly=True, subdir=False, lock=False)

    with env.begin() as txn:
        num_entries = txn.stat()['entries']
        print(f"Processing {lmdb_file}: {num_entries} entries")

        cursor = txn.cursor()

        for key, value in cursor:
            try:
                # load pytorch data obj manually due to the imcompatible PyG version of OC22 dataset
                data = Data.from_dict(pickle.loads(value).__dict__)

                # get target value and sid from training dataset
                sid = data.sid
                y_relaxed = data.y_relaxed

                row = {
                    "sid": sid,
                    "y_relaxed": y_relaxed
                }

                # grab rest of the data from metadata
                if sid in metadata:
                    row.update(metadata[sid])

                all_data.append(row)
            except:
                pass
    env.close()

df = pd.DataFrame(all_data, columns=columns)
df.to_csv("training_FULL.csv", index=False)

Processing oc22/is2re-total/train/data.0000.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0001.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0002.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0003.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0004.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0005.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0006.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0007.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0008.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0009.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0010.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0011.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0012.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0013.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0014.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0015.lmdb: 1120 

In [34]:
# get the samples with H as adsorbate
# creates a mask where True if ads is H
contains_h_adsorbate = (df['ads_symbols'] == 'H') & (df['nads'] == 1)

# robust mask for rows with no adsorbates: handles None/NaN, empty lists, and stringified empty lists like '[]'
def _is_no_ads(x):
    # None or NaN
    if pd.isna(x):
        return True
    
    # actual empty list/tuple/set
    if isinstance(x, (list, tuple, set)):
        return len(x) == 0
    
    # string forms: '', '[]', , 'None'
    s = str(x).strip()
    if s == '' or s == '[]' or s.lower() == 'none':
        return True
    
    return False

# creates a mask where False if ads is None
contains_no_ads = df['ads_symbols'].apply(_is_no_ads)

df_only_h = df[contains_h_adsorbate]
df_only_h = df_only_h.reset_index(drop=True)

df_no_ads = df[contains_no_ads]
df_no_ads = df_no_ads.reset_index(drop=True)

df_only_h.to_csv("training_FULL_h.csv", index=False)
df_no_ads.to_csv("training_FULL_no_ads.csv", index=False)

# --------------------------------------------------------------
# randomly sample 100 data points from each dataset
# for the purpose of testing the featurizer. 
# remember to remove this section later

df_only_h_sample = df_only_h.sample(n=100)
df_no_ads_sample = df_no_ads.sample(n=100)
df_only_h_sample.to_csv("training_SAMPLE_h.csv", index=False)
df_no_ads_sample.to_csv("training_SAMPLE_no_ads.csv", index=False)

print(f"size of entire dataset: {len(df)}")
print(f"size of H-ads dataset: {len(df_only_h)}")
print(f"size of no-ads dataset: {len(df_no_ads)}")
print(f"sample size: n = {len(df_only_h_sample)}")

size of entire dataset: 45890
size of H-ads dataset: 1605
size of no-ads dataset: 14646
sample size: n = 100


In [ ]:
# featurization
# please specify whether to featurize sample or full dataset:
h = df_only_h_sample # use df_only_h to process full dataset
no_ads = df_no_ads_sample # use df_no_ads to process full dataset

h_clean = h[['bulk_id', 'bulk_symbols', 'y_relaxed']]
no_ads_clean = no_ads[['bulk_id', 'bulk_symbols', 'y_relaxed']]

print(h_clean.head())
print(no_ads_clean.head())

       bulk_id bulk_symbols   y_relaxed
1626  mp-37436     Li7Sb1O6 -478.520592
3027  mp-29680    Ta4Bi8O22 -777.683692
1465  mp-18994    Cs4Mn4O16 -254.609219
67    mp-29164    Sr4Bi4O12 -681.089878
1163   mp-9200      K3Au1O1 -148.816550
          bulk_id bulk_symbols    y_relaxed
4580   mp-1227338     Be6Zn2O8  -538.718979
384      mp-10418     Rb4Sb4O8  -301.345122
2318    mp-759541    Ba4Pd8O16  -498.813263
11614     mp-5478   Tl8Pd24O32  -328.306257
3737    mp-761148   Ti18Sn2O40 -1024.342276


In [5]:
# convert bulk symbols to Composition object
from pymatgen.core import Composition

pd.options.mode.copy_on_write = True
h_clean['bulk_symbols'] = h_clean['bulk_symbols'].apply(Composition)
no_ads_clean['bulk_symbols'] = no_ads_clean['bulk_symbols'].apply(Composition)

print("---training set with H adsorbate---")
for c in h_clean['bulk_symbols'].head():
    print(f"formula: {c.formula}; num of atoms: {c.num_atoms}")

print("---training set surface only---")
for c in no_ads_clean['bulk_symbols'].head():
    print(f"formula: {c.formula}; num of atoms: {c.num_atoms}")

---training set with H adsorbate---
formula: Li7 Sb1 O6; num of atoms: 14.0
formula: Ta4 Bi8 O22; num of atoms: 34.0
formula: Cs4 Mn4 O16; num of atoms: 24.0
formula: Sr4 Bi4 O12; num of atoms: 20.0
formula: K3 Au1 O1; num of atoms: 5.0
---training set surface only---
formula: Be6 Zn2 O8; num of atoms: 16.0
formula: Rb4 Sb4 O8; num of atoms: 16.0
formula: Ba4 Pd8 O16; num of atoms: 28.0
formula: Tl8 Pd24 O32; num of atoms: 64.0
formula: Ti18 Sn2 O40; num of atoms: 60.0


In [ ]:
from matminer.featurizers.composition import WenAlloys
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ElementFraction
from matminer.featurizers.composition import TMetalFraction
from matminer.featurizers.composition import Stoichiometry
from matminer.featurizers.composition import BandCenter

def featurize_df(df):
    featurized_df = df.copy()
    print("Initial df shape:", featurized_df.shape)
    
    features = [
        'AtomicWeight',
        'MeltingT',
        'CovalentRadius',
        'Electronegativity',
        'NsValence',
        'NpValence',
        'NdValence',
        'NfValence',
        'NValence',
        'NsUnfilled',
        'NpUnfilled',
        'NdUnfilled',
        'NfUnfilled',
        'NUnfilled',
        'GSvolume_pa',
        'GSbandgap',
        'GSmagmom'
    ]
    
    f = ElementProperty('magpie', features=features, stats=['mean'], impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True, return_errors=True)
    print("After ElementProperty:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = WenAlloys(impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After WenAlloys:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = ElementFraction()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After ElementFraction:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = TMetalFraction()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After TMetalFraction:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = Stoichiometry()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After Stoichiometry:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = BandCenter(impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After BandCenter:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    return featurized_df

In [9]:
import warnings


# suppress annoying warnings
warnings.filterwarnings('ignore', message='.*impute_nan.*')

# warning: this step can take a while!
h_featurized = featurize_df(h_clean)
no_ads_featurized = featurize_df(no_ads_clean)

Initial df shape: (100, 3)
Starting featurization of 100 rows...
n_jobs set to: 1


ElementProperty:   0%|          | 0/100 [00:00<?, ?it/s]

After ElementProperty: <class 'pandas.core.frame.DataFrame'> (100, 21)


WenAlloys:   0%|          | 0/100 [00:00<?, ?it/s]

After WenAlloys: <class 'pandas.core.frame.DataFrame'> (100, 46)


...

After TMetalFraction: <class 'pandas.core.frame.DataFrame'> (100, 165)


Stoichiometry:   0%|          | 0/100 [00:00<?, ?it/s]

After Stoichiometry: <class 'pandas.core.frame.DataFrame'> (100, 171)


BandCenter:   0%|          | 0/100 [00:00<?, ?it/s]

After BandCenter: <class 'pandas.core.frame.DataFrame'> (100, 172)


In [10]:
h_featurized.to_csv("training_SAMPLE_h_comp_featurized_.csv")
no_ads_featurized.to_csv("training_SAMPLE_no_ads_comp_featurized_.csv")

possible MP fields that can be requested by API:

['builder_meta',
 'nsites',
 'elements',
 'nelements',
 'composition',
 'composition_reduced',
 'formula_pretty',
 'formula_anonymous',
 'chemsys',
 'volume',
 'density',
 'density_atomic',
 'symmetry',
 'deprecated',
 'deprecation_reasons',
 'last_updated',
 'origins',
 'warnings',
 'property_name',
 'task_ids',
 'uncorrected_energy_per_atom',
 'formation_energy_per_atom',
 'energy_above_hull',
 'is_stable',
 'equilibrium_reaction_energy_per_atom',
 'decomposes_to',
 'xas',
 'grain_boundaries',
 'band_gap',
 'cbm',
 'vbm',
 'efermi',
 'is_gap_direct',
 'is_metal',
 'es_source_calc_id',
 'bandstructure',
 'dos',
 'dos_energy_up',
 'dos_energy_down',
 'is_magnetic',
 'ordering',
 'total_magnetization',
 'total_magnetization_normalized_vol',
 'total_magnetization_normalized_formula_units',
 'num_magnetic_sites',
 'num_unique_magnetic_sites',
 'types_of_magnetic_species',
 'bulk_modulus',
 'shear_modulus',
 'universal_anisotropy',
 'homogeneous_poisson',
 'e_total',
 'e_ionic',
 'e_electronic',
 'n',
 'e_ij_max',
 'weighted_surface_energy_EV_PER_ANG2',
 'weighted_surface_energy',
 'weighted_work_function',
 'surface_anisotropy',
 'shape_factor',
 'has_reconstructed',
 'possible_species',
 'has_props',
 'theoretical',
 'database_IDs']

DensityFeatures:
- density
- vpa
- packing fraction

GlobalSymmetryFeatures:
- spacegroup_num
- crystal_system
- crystal_system_int
- is_centrosymmetric
- n_symmetry_ops


StructuralHeterogeneity
StructuralComplexity
??

In [24]:
# get structure properties
from mp_api.client import MPRester
from pymatgen.core import Structure
from matminer.featurizers.structure import DensityFeatures, GlobalSymmetryFeatures, EwaldEnergy


key = "H1TcFfkjPB8IR4vjQB1vXdZ7CbceH5I4"

density_featurizer = DensityFeatures()
global_symmetry_featurizer = GlobalSymmetryFeatures()
ewald_energy_featurizer = EwaldEnergy()

def featurize_structure(df):
    structure_features = []
    structure_features_cols = [
        'bulk_symbols',
        'density', 
        'vpa', 
        'packing_fraction', 
        'spacegroup_num', 
        'crystal_system', 
        'crystal_system_int', 
        'is_centrosymmetric', 
        'n_symmetry_ops']
    
    with MPRester(key) as mpr:
        for i, row in df.iterrows():
            bulk_symbol = row['bulk_symbols'] # already a Composition object
            formula = bulk_symbol.reduced_formula

            try:
                # search for material structure
                docs = mpr.materials.summary.search(formula=formula, fields=['material_id', 'structure', 'energy_per_atom'])
                if docs:
                    # get the structure with the lowest energy/atom
                    best_doc = min(docs, key=lambda x: x.energy_per_atom)
                    best_structure = best_doc.structure

                    # generate structure features
                    density_features = density_featurizer.featurize(best_structure)
                    global_symmetry_features = global_symmetry_featurizer.featurize(best_structure)

                    # store generated features
                    structure_features.append([bulk_symbol] + density_features + global_symmetry_features)
                else:
                    # add 0 if structure cannot be found
                    structure_features.append([bulk_symbol] + ([None] * 8))
            except:
                pass

    return pd.DataFrame(data=structure_features, columns=structure_features_cols)


In [25]:
h_structure_featurized = featurize_structure(h_clean)
no_ads_structure_featurized = featurize_structure(no_ads_clean)

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/18 [00:00<?, ?it/s]

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
h_structure_featurized


,bulk_symbols,density,vpa,packing_fraction,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops
0,"(Li, Sb, O)",3.544145,8.913568,0.862161,1.0,triclinic,7.0,False,2.0
1,"(Ta, Bi, O)",9.283679,14.454630,0.423727,2.0,triclinic,7.0,True,4.0
2,"(Cs, Mn, O)",3.649580,19.097724,0.774397,62.0,orthorhombic,5.0,True,16.0
3,"(Sr, Bi, O)",7.355103,15.559794,0.686152,14.0,monoclinic,6.0,True,8.0
4,"(K, Au, O)",3.845922,28.519092,1.016985,221.0,cubic,1.0,True,96.0
...,...,...,...,...,...,...,...,...,...
95,"(Mn, Sn, O)",NaN,NaN,NaN,NaN,None,NaN,None,NaN
96,"(Cr, Au, O)",NaN,NaN,NaN,NaN,None,NaN,None,NaN
97,"(Y, Au, O)",7.993944,16.507421,0.553457,194.0,hexagonal,2.0,True,48.0
98,"(Ca, Os, O)",7.839020,11.790716,0.616521,62.0,orthorhombic,5.0,True,8.0


In [27]:
h_featurized

,bulk_id,bulk_symbols,y_relaxed,MagpieData mean AtomicWeight,MagpieData mean MeltingT,MagpieData mean CovalentRadius,MagpieData mean Electronegativity,MagpieData mean NsValence,MagpieData mean NpValence,MagpieData mean NdValence,...,Ts,Og,transition metal fraction,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,band center
1626,mp-37436,"(Li, Sb, O)",-478.520592,19.024529,314.886429,102.214286,2.110714,1.500000,1.928571,0.714286,...,0,0,0.000000,3,0.662401,0.588755,0.539514,0.521344,0.509793,4.612597
3027,mp-29680,"(Ta, Bi, O)",-777.683692,80.812398,550.611765,97.529412,2.877647,2.000000,3.294118,2.705882,...,0,0,0.117647,3,0.698491,0.658522,0.647905,0.647137,0.647061,6.087441
1465,mp-18994,"(Cs, Mn, O)",-254.609219,41.973516,339.965000,107.833333,2.683333,1.833333,2.666667,0.833333,...,0,0,0.166667,3,0.707107,0.673540,0.666927,0.666678,0.666667,5.450709
67,mp-29164,"(Sr, Bi, O)",-681.089878,68.919720,351.760000,108.200000,2.658000,2.000000,3.000000,2.000000,...,0,0,0.000000,3,0.663325,0.614463,0.600984,0.600078,0.600002,5.508122
1163,mp-9200,"(K, Au, O)",-148.816550,66.052174,480.344000,162.200000,1.688000,1.200000,0.800000,2.000000,...,0,0,0.000000,3,0.663325,0.614463,0.600984,0.600078,0.600002,3.614867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3756,MnSnO4-rutile,"(Mn, Sn, O)",-484.053953,39.607608,373.880000,90.333333,2.878333,2.000000,3.000000,2.500000,...,0,0,0.166667,3,0.707107,0.673540,0.666927,0.666678,0.666667,6.085708
2817,CrAuO4-rutile,"(Cr, Au, O)",-519.243940,52.160045,622.755000,89.833333,2.993333,1.666667,2.666667,2.500000,...,0,0,0.166667,3,0.707107,0.673540,0.666927,0.666678,0.666667,6.408714
3048,mp-30251,"(Y, Au, O)",-562.497070,79.467805,811.482500,114.500000,2.660000,1.750000,2.000000,2.750000,...,0,0,0.000000,3,0.612372,0.538609,0.506099,0.501109,0.500098,5.719124
1547,mp-1016900,"(Ca, Os, O)",-193.658560,55.661240,917.080000,103.600000,2.704000,2.000000,2.400000,1.200000,...,0,0,0.200000,3,0.663325,0.614463,0.600984,0.600078,0.600002,5.779753


In [30]:
# h_final = pd.concat([h_featurized, h_structure_featurized], axis=1)
# no_ads_final = pd.concat([no_ads_featurized, no_ads_structure_featurized], axis=1)

h_final = pd.merge(h_featurized, h_structure_featurized, on='bulk_symbols')
no_ads_final = pd.merge(no_ads_featurized, no_ads_structure_featurized, on='bulk_symbols')


# here save the final fully featurized dataset.
# rename the file if this is the full dataset.
# remember to check the dataset one more time when implementing the ML pipeline.
h_final.to_csv("training_SAMPLE_h_featurized.csv", index=False)
no_ads_final.to_csv("training_SAMPLE_no_ads_featurized.csv", index=False)

In [ ]:
h_final

,bulk_id,bulk_symbols,y_relaxed,MagpieData mean AtomicWeight,MagpieData mean MeltingT,MagpieData mean CovalentRadius,MagpieData mean Electronegativity,MagpieData mean NsValence,MagpieData mean NpValence,MagpieData mean NdValence,...,10-norm,band center,density,vpa,packing_fraction,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops
0,mp-37436,"(Li, Sb, O)",-478.520592,19.024529,314.886429,102.214286,2.110714,1.500000,1.928571,0.714286,...,0.509793,4.612597,3.544145,8.913568,0.862161,1.0,triclinic,7.0,False,2.0
1,mp-29680,"(Ta, Bi, O)",-777.683692,80.812398,550.611765,97.529412,2.877647,2.000000,3.294118,2.705882,...,0.647061,6.087441,9.283679,14.454630,0.423727,2.0,triclinic,7.0,True,4.0
2,mp-18994,"(Cs, Mn, O)",-254.609219,41.973516,339.965000,107.833333,2.683333,1.833333,2.666667,0.833333,...,0.666667,5.450709,3.649580,19.097724,0.774397,62.0,orthorhombic,5.0,True,16.0
3,mp-29164,"(Sr, Bi, O)",-681.089878,68.919720,351.760000,108.200000,2.658000,2.000000,3.000000,2.000000,...,0.600002,5.508122,7.355103,15.559794,0.686152,14.0,monoclinic,6.0,True,8.0
4,mp-9200,"(K, Au, O)",-148.816550,66.052174,480.344000,162.200000,1.688000,1.200000,0.800000,2.000000,...,0.600002,3.614867,3.845922,28.519092,1.016985,221.0,cubic,1.0,True,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,MnSnO4-rutile,"(Mn, Sn, O)",-484.053953,39.607608,373.880000,90.333333,2.878333,2.000000,3.000000,2.500000,...,0.666667,6.085708,NaN,NaN,NaN,NaN,None,NaN,None,NaN
104,CrAuO4-rutile,"(Cr, Au, O)",-519.243940,52.160045,622.755000,89.833333,2.993333,1.666667,2.666667,2.500000,...,0.666667,6.408714,NaN,NaN,NaN,NaN,None,NaN,None,NaN
105,mp-30251,"(Y, Au, O)",-562.497070,79.467805,811.482500,114.500000,2.660000,1.750000,2.000000,2.750000,...,0.500098,5.719124,7.993944,16.507421,0.553457,194.0,hexagonal,2.0,True,48.0
106,mp-1016900,"(Ca, Os, O)",-193.658560,55.661240,917.080000,103.600000,2.704000,2.000000,2.400000,1.200000,...,0.600002,5.779753,7.839020,11.790716,0.616521,62.0,orthorhombic,5.0,True,8.0
